In [1]:
!pip install pandas
!pip install numpy
!pip install sqlalchemy
!pip install psycopg2
!pip install sqlalchemy
!pip install ipython-sql

  Using cached psycopg2-2.9.3-cp38-cp38-win_amd64.whl (1.1 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\julia\\anaconda3\\Lib\\site-packages\\psycopg2\\_psycopg.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
#IMPORTS
import pandas as pd
import json
import requests
import numpy as np
from sqlalchemy import create_engine
import psycopg2
%load_ext sql
import sqlalchemy

In [3]:
#CONECTANDO AO BANCO DE DADOS
%sql sqlite:///sqlite3.db
engine = create_engine('sqlite:///sqlite3.db')

In [4]:
#TRANSFORMANDO O CSV EM DATAFRAME
df_compras=pd.read_csv('https://us-central1-raccoon-bi.cloudfunctions.net/psel_de_compras')


In [6]:
df_compras

,nome,show,gastos
0,Joseph Wray,Symbolic Toy,3142.35
1,Barbara Mcneil,Symbolic Toy,2431.78
2,Eduardo Espenshade,Symbolic Toy,386.32
3,Lucia Hayes,Symbolic Toy,3373.49
4,Johnny Hairston,Symbolic Toy,2805.88
...,...,...,...
495,Cassandra Gonsalves,Terrestrial Chair,154.39
496,Christopher Mueller,Symbolic Toy,1797.44
497,Christopher Mueller,Nascent Letter,1511.91
498,Christopher Mueller,Nascent Letter,3439.69


In [7]:
#TRANSFORMANDO O JSON EM DATAFRAME
df_ingressos=pd.read_json('https://us-central1-raccoon-bi.cloudfunctions.net/psel_de_ingressos')
df_ingressos

,ano,dia,mes,nome,status,tipo
0,2020,1,1,Benjamin Mcdonald,Nao Concluido,VIP
1,2020,1,1,Mitchell Kostyla,Nao Concluido,VIP
2,2020,1,1,Gerald Allen,Nao Concluido,VIP
3,2020,1,1,Frederick Bank,Problema no Pagamento,VIP
4,2020,1,1,Tina Yee,Nao Concluido,VIP
...,...,...,...,...,...,...
95,2020,3,1,Richard Jimenez,Nao Concluido,Pista Premium
96,2020,3,1,Curtis Hernandez,Problema no Pagamento,Pista Premium
97,2020,3,1,Alice Gamba,Concluido,Pista Premium
98,2020,3,1,Brenda Kozak,Problema no Pagamento,Pista Premium


In [8]:
#TRANSFORMANDO O JSON EM DATAFRAME
df_shows=pd.read_json('https://us-central1-raccoon-bi.cloudfunctions.net/psel_de_shows')
df_shows

,Nascent Letter,Symbolic Toy,Terrestrial Chair
ano,2020,2020,2020
dia,2,3,1
mes,1,1,1


In [9]:
try:
    df_compras.to_sql('compras', con=engine)
    df_ingressos.to_sql('ingressos', con=engine)
except:
    print('Tabela ja exite!')


Tabela ja exite!


### 1. Qual a média de gastos de pessoas com ingresso Pista? #

In [10]:
'''
#USANDO PANDAS
'''

filtering=df_ingressos['nome'][(
    df_ingressos.status=='Concluido')&(
    df_ingressos.tipo=='Pista')]

df_pista=df_compras['nome'].isin(filtering)
df_compras.loc[df_pista,'gastos'].mean()

1935.9419444444445

In [11]:
'''
# USANDO SQL
'''

%sql select AVG(gastos) from compras WHERE nome IN (select nome from ingressos where status='Concluido' AND tipo='Pista') 

 * sqlite:///sqlite3.db
Done.


AVG(gastos)
1935.9419444444445


### 2. Quais pessoas não compareceram aos shows?#

In [12]:
'''
#USANDO PANDAS
'''

df_presenca=df_compras['nome']
df_s_presenca=df_ingressos[~df_ingressos['nome'].isin(df_presenca)].sort_values(by='nome', ascending=True)
df_s_presenca=df_s_presenca.drop_duplicates(subset=['nome'])
df_s_presenca

,ano,dia,mes,nome,status,tipo
97,2020,3,1,Alice Gamba,Concluido,Pista Premium
88,2020,2,1,Amy Mejia,Problema no Pagamento,Pista Premium
76,2020,1,1,Brenda Key,Concluido,Pista Premium
98,2020,3,1,Brenda Kozak,Problema no Pagamento,Pista Premium
53,2020,2,1,Courtney Edwards,Concluido,Pista
96,2020,3,1,Curtis Hernandez,Problema no Pagamento,Pista Premium
12,2020,1,1,Deborah Silsby,Concluido,VIP
14,2020,1,1,Donna Chandler,Nao Concluido,VIP
62,2020,3,1,Edward Smith,Problema no Pagamento,Pista
46,2020,1,1,Elizabeth Daly,Problema no Pagamento,Pista


### 2. Quais pessoas não compareceram aos shows? #

In [13]:
'''
#USANDO SQL
'''
%sql select  DISTINCT(nome) from ingressos where nome NOT IN (select nome from compras) ORDER BY nome ASC

 * sqlite:///sqlite3.db
Done.


nome
Alice Gamba
Amy Mejia
Brenda Key
Brenda Kozak
Courtney Edwards
Curtis Hernandez
Deborah Silsby
Donna Chandler
Edward Smith
Elizabeth Daly


### 3. Quais pessoas compraram ingressos com concorrentes? #

In [14]:
''' 
#USANDO PANDAS
'''
vendidos=df_ingressos['nome'][(
    df_ingressos.status=='Concluido')]


df_concorrente=df_compras[~df_compras['nome'].isin(vendidos)].sort_values(by='nome', ascending=True)
df_concorrente.drop_duplicates(['nome'])


,nome,show,gastos
285,Adelina Thews,Terrestrial Chair,2372.34
48,Alan Gravois,Nascent Letter,463.15
112,Albert Fadden,Terrestrial Chair,1637.19
118,Alberto Gumm,Terrestrial Chair,2297.47
224,Allen Bylsma,Symbolic Toy,454.92
...,...,...,...
83,William Converse,Nascent Letter,1627.07
422,William Cotton,Symbolic Toy,917.68
117,William Polite,Terrestrial Chair,304.31
355,William Swann,Terrestrial Chair,2962.95


In [15]:
'''
#USANDO SQL
'''
%sql select DISTINCT(nome) from compras where nome NOT IN (select nome from ingressos where status=='Concluido') ORDER BY nome ASC

 * sqlite:///sqlite3.db
Done.


nome
Adelina Thews
Alan Gravois
Albert Fadden
Alberto Gumm
Allen Bylsma
Allison Magill
Amanda Jones
Andrea Galyen
Andrew Hoff
Anna Hines


### 4. Qual o dia com maior gasto? #

In [16]:
'''
#USANDO PANDAS
'''

df_compras.groupby(['show']).sum()

,gastos
show,
Nascent Letter,269765.88
Symbolic Toy,330810.92
Terrestrial Chair,319930.31


In [17]:
'''
#USANDO SQL
'''

%sql select show,SUM(gastos) as Total_Gastos from compras GROUP BY show 

 * sqlite:///sqlite3.db
Done.


show,Total_Gastos
Nascent Letter,269765.8799999999
Symbolic Toy,330810.92
Terrestrial Chair,319930.3100000002


### 5. Faça uma lista com os clientes que desistiram de comprar o ingresso com a AT, a soma do valor que foi gasto durante os shows e quais shows eles desistiram de comprar. #

In [18]:
df_merge=pd.merge(df_compras,df_ingressos,how='inner',on='nome',)
df_merge=df_merge[df_merge.status!='Concluido']
final_df=df_merge.groupby('nome').agg({'show':lambda x:list(x),'gastos':'sum'})
final_df


,show,gastos
nome,,
Andrea Galyen,"[Symbolic Toy, Symbolic Toy, Terrestrial Chair...",10267.62
Benjamin Mcdonald,[Terrestrial Chair],2309.81
Blanche Knox,"[Nascent Letter, Terrestrial Chair]",3500.72
Charles Hershenson,"[Nascent Letter, Terrestrial Chair]",3500.73
Daniel Puckett,"[Symbolic Toy, Symbolic Toy, Terrestrial Chair...",5470.98
Debra Howard,"[Symbolic Toy, Symbolic Toy, Nascent Letter, N...",22636.04
Donna Wal,"[Symbolic Toy, Symbolic Toy, Nascent Letter, N...",11954.46
Edna Martinez,"[Symbolic Toy, Symbolic Toy, Symbolic Toy, Sym...",41205.54
Emil Nord,"[Symbolic Toy, Symbolic Toy, Nascent Letter, N...",8352.80


In [19]:
lista_desistencia=[]
for cliente in final_df.itertuples():
    case={'nome':cliente[0],'gastos':round(cliente[2], 2),'show':cliente[1]}
    lista_desistencia.append(case)

#print(lista_desistencia)


In [20]:
for client in lista_desistencia:
    print(client)


{'nome': 'Andrea Galyen', 'gastos': 10267.62, 'show': ['Symbolic Toy', 'Symbolic Toy', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair']}
{'nome': 'Benjamin Mcdonald', 'gastos': 2309.81, 'show': ['Terrestrial Chair']}
{'nome': 'Blanche Knox', 'gastos': 3500.72, 'show': ['Nascent Letter', 'Terrestrial Chair']}
{'nome': 'Charles Hershenson', 'gastos': 3500.73, 'show': ['Nascent Letter', 'Terrestrial Chair']}
{'nome': 'Daniel Puckett', 'gastos': 5470.98, 'show': ['Symbolic Toy', 'Symbolic Toy', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair']}
{'nome': 'Debra Howard', 'gastos': 22636.04, 'show': ['Symbolic Toy', 'Symbolic Toy', 'Nascent Letter', 'Nascent Letter', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair', 'Terrestrial Chair']}
{'nome': 'Donna Wal', 'gastos': 11954.46, 'show': ['Symbolic Toy', 'Symbolic Toy', 'Nascent Letter', 'Nascent Lett

# JULIANO XAVIER ZYCZYCKI
